### The Agentic Heatbeat Pattern

This notebook demonstrates the [Agentic Heartbeat pattern](https://github.com/marcilio/AgenticHeartbeatPattern) through a practical “Status Report” example.

Author: Marcilio Mendonca (https://www.linkedin.com/in/marcilio/)

Aug 2025

In [ ]:
%load_ext autoreload
%autoreload 2
%pip install strands-agents==1.3.0 strands-agents-tools==0.2.3 python-dotenv

In [ ]:
# Load AWS credentials from a file (file not provided :P)
from dotenv import load_dotenv
load_dotenv('../.aws-creds')


In [ ]:
hierarchical_model = {
    "CEO": ["VP of Engineering", "VP of Marketing", "VP of Sales"],
    "VP of Engineering": ["Director of Platform Engineering", "Director of Machine Learning"],
    "VP of Marketing": ["Director of Brand Strategy", "Director of Digital Campaigns"],
    "VP of Sales": ["Director of Enterprise Accounts", "Director of Small Business Sales"],
    "Director of Platform Engineering": ["Senior Backend Engineer", "Cloud Infrastructure Engineer"],
    "Director of Machine Learning": ["Machine Learning Engineer", "Data Scientist"],
    "Director of Brand Strategy": ["Creative Copywriter", "Visual Designer"],
    "Director of Digital Campaigns": ["SEO Specialist", "Social Media Manager"],
    "Director of Enterprise Accounts": ["Enterprise Account Executive", "Solutions Architect"],
    "Director of Small Business Sales": ["Sales Development Representative", "Account Manager"]
}

status_updates = {
    "Senior Backend Engineer": {
        "highlights": [
            "Successfully refactored the authentication module, improving response time by 20%.",
            "Reduced API error rate by 15% through better exception handling.",
            "Completed code review for two critical feature branches ahead of schedule."
        ],
        "lowlights": [
            "Blocked on integration tests due to missing mock service.",
            "Legacy code dependencies caused delays in feature rollout."
        ]
    },
    "Cloud Infrastructure Engineer": {
        "highlights": [
            "Completed Terraform automation for staging environment.",
            "Optimized cloud resource usage, reducing costs by 10%.",
            "Improved monitoring alerts, decreasing false positives."
        ],
        "lowlights": [
            "Experiencing intermittent failures with the CI/CD pipeline during deploys.",
            "Slow response to an outage due to incomplete runbooks."
        ]
    },
    "Machine Learning Engineer": {
        "highlights": [
            "Deployed a new model for customer churn prediction with 5% higher accuracy.",
            "Improved model training pipeline, reducing training time by 25%.",
            "Collaborated with data scientists to refine feature selection."
        ],
        "lowlights": [
            "Data pipeline for model training is delayed due to upstream changes.",
            "Model drift detected; retraining required sooner than expected."
        ]
    },
    "Data Scientist": {
        "highlights": [
            "Published the quarterly data insights report, well-received by execs.",
            "Developed new dashboard visualizations to track customer engagement.",
            "Provided actionable insights that influenced product roadmap decisions."
        ],
        "lowlights": [
            "Some dashboard metrics are outdated due to missing ETL job.",
            "Data quality issues delayed a key analysis project."
        ]
    },
    "Creative Copywriter": {
        "highlights": [
            "Delivered compelling content for the new product landing page.",
            "Created copy for a successful email marketing campaign.",
            "Collaborated with design team to align messaging and visuals."
        ],
        "lowlights": [
            "Stuck waiting for product team input on new feature descriptions.",
            "Tight deadlines caused rushed copy with minor errors."
        ]
    },
    "Visual Designer": {
        "highlights": [
            "Completed new icon set and brand guidelines for mobile app redesign.",
            "Designed graphics for a high-profile marketing campaign.",
            "Improved UI consistency across product pages."
        ],
        "lowlights": [
            "Last-minute design requests are impacting sprint commitments.",
            "Resource constraints delayed delivery of some assets."
        ]
    },
    "SEO Specialist": {
        "highlights": [
            "Top 5 keywords are ranking on page 1 after optimization.",
            "Increased organic search traffic by 12% month-over-month.",
            "Improved site speed resulting in better search rankings."
        ],
        "lowlights": [
            "Traffic dropped from a recent algorithm change – under investigation.",
            "Keyword competition increased in key markets."
        ]
    },
    "Social Media Manager": {
        "highlights": [
            "Launched successful campaign with 30% increase in engagement.",
            "Grew follower base by 8% in the last month.",
            "Partnered with influencers for brand awareness boosts."
        ],
        "lowlights": [
            "Received negative feedback on a post – crisis management in progress.",
            "Platform API changes disrupted scheduled posts."
        ]
    },
    "Enterprise Account Executive": {
        "highlights": [
            "Closed a $500k deal with a Fortune 500 client.",
            "Expanded existing accounts leading to 20% revenue growth.",
            "Delivered strong quarterly sales presentations."
        ],
        "lowlights": [
            "Delays in legal review are slowing down another key contract.",
            "Long sales cycles impacting quarterly targets."
        ]
    },
    "Solutions Architect": {
        "highlights": [
            "Presented a successful proof of concept to a strategic client.",
            "Documented scalable architecture patterns for internal use.",
            "Provided critical technical support during RFP process."
        ],
        "lowlights": [
            "Blocked by internal tooling limitations for demo scalability.",
            "Difficulty aligning architecture with legacy systems."
        ]
    },
    "Sales Development Representative": {
        "highlights": [
            "Set a record number of qualified meetings this week.",
            "Increased outreach efficiency with new email templates.",
            "Achieved highest conversion rate in the team."
        ],
        "lowlights": [
            "CRM data inconsistency caused some misrouted leads.",
            "Facing challenges with prospect contact availability."
        ]
    },
    "Account Manager": {
        "highlights": [
            "Achieved 95% customer satisfaction on quarterly NPS survey.",
            "Successfully upsold new features to key clients.",
            "Resolved high priority support issues promptly."
        ],
        "lowlights": [
            "Facing churn risk with a major client due to support issues.",
            "Resource limitations impacting proactive client engagement."
        ]
    }
}


In [ ]:
import os
from datetime import datetime

logs_path = "logs"

def logging_handler(function_role: str, *args, **kwargs):
    message = kwargs.get("message")
    if not message:
        return

    # Only log assistant messages with content & text
    if message.get("role") == "assistant" and "content" in message:
        # Extract all "text" fields
        texts = [
            block.get("text")
            for block in message["content"]
            if isinstance(block, dict) and "text" in block
        ]

        if texts:  # Only proceed if we found any text
            markdown_text = "\n".join(texts)
            log_entry = f"## {datetime.now().strftime('%Y-%m-%d %H:%M:%S')} - {function_role}\n{markdown_text}\n\n"

            with open(os.path.join(logs_path, f"{function_role}.md"), "a", encoding="utf-8") as f:
                f.write(log_entry)


def status_report_to_markdown(report: dict) -> str:
    md_parts = []

    if "highlights" in report and report["highlights"]:
        md_parts.append("## Highlights")
        for item in report["highlights"]:
            md_parts.append(f"- {item}")

    if "lowlights" in report and report["lowlights"]:
        md_parts.append("\n## Lowlights")
        for item in report["lowlights"]:
            md_parts.append(f"- {item}")

    return "\n".join(md_parts)


In [ ]:
from strands import Agent, tool

model_id = "us.anthropic.claude-3-7-sonnet-20250219-v1:0"

@tool
def get_direct_reports(function_role: str):
    """
    Returns the name of the direct reports for the given function role.

    Input: str
    - the name of the function role

    Output: List[str]
    - list with the names of the direct reports of a given function role

    Example:
    - Input: "VP of Engineering"
    - Output: ["Director of Platform Engineering", "Director of Machine Learning"]
    """
    print(f"\nTOOL: Getting the list of direct reports for {function_role}")
    all_function_roles = set(node for pair in hierarchical_model.items() for node in [pair[0]] + pair[1])
    if function_role not in all_function_roles:
        return f"Error! Invalid function role: {function_role}. Must be one of {list(all_function_roles)}"
    if function_role not in hierarchical_model:
        return "You don't have any direct reports"
    return f"These are your direct reports: {','.join(hierarchical_model[function_role])}"

@tool
def produce_status_report(function_role: str):
    """
    Roles with no direct reports can use this tool to produce their status report (function_role).

    Input: str
    - the name of the function role that has no direct reports

    Output: str
    - the status report in markdown

    Example:
    - Input: "Account Manager"
    - Output:
        ## Highlights
        - Highlight 1
        - Highlight 2
        ## Lowlights
        - Lowlight 1
        - Lowlight 2
    """
    print(f"\nTOOL: Producing status report for {function_role}")
    if function_role not in status_updates:
        return f"No status report exists for role {function_role}."
    return status_report_to_markdown(status_updates[function_role])


@tool
def request_status_report_from(function_role: str):
    """
    Managers can use this tool to request the status report from their direct reports (function_role).

    Input: str
    - the name of the function role

    Output: str
    - the status report in markdown

    Example:
    - Input: "Director of Platform Engineering"
    - Output:
        ## Highlights
        - Highlight 1
        - Highlight 2
        ## Lowlights
        - Lowlight 1
        - Lowlight 2
    """
    print(f"\nTOOL: Getting status reports from directs of {function_role}")
    status_report_agent = Agent(
        agent_id=function_role,
        model=model_id,
        system_prompt="""
            Your work for the HeartBeart AI Agent organization.
            Your goal is to write a status report to your manager or board of directors if you are the CEO.
            The status report consists of a list of highlights and lowlights for the teams and people you manage.

            You must follow this plan to build your own status report:
            1- get the list of your direct reports
            2a- if you don't have any direct reports, you need to produce your own status report
            2b- if you have direct reports, you need to request their status reports, then summarize their highlights and lowlights into your own status report. Use your own words. Focus on the most relevant piece of information.
            3- Return your status report in markdown like this:
                ## Highlights
                - Highlight 1
                - Highlight 2
                ## Lowlights
                - Lowlight 1
                - Lowlight 2
            Use the tools at your disposal to produce the status report.
        """,
        callback_handler = lambda *args, **kwargs: logging_handler(function_role, *args, **kwargs),
        tools=[get_direct_reports, produce_status_report, request_status_report_from],
    )
    return status_report_agent(f"Your role in the organization is {function_role}.")

# get_direct_reports("Director of Small Business Sales")
status_report = request_status_report_from("CEO")
status_report
